In [ ]:
import numpy as np
import matplotlib.pyplot as pl

import torch
from masking import *

import csv

from latencies import *

In [ ]:
fs=48828

I0 = 100 +10 - 32 #intensity ref for masker    #100 dB rms=1  +10 dB amp 5   (/sqrt(2))   #-32 dB masker atten
I0 = 10*np.log10( 10**(I0/10)/(fs/2) ) #spectral density

#Note for later: import common script to analysis instead?

### First estimation Q10 

In [ ]:
def lin_reg(x, y):
    x0=np.mean(x)
    y0=np.mean(y)
    x2=x-x0
    y2=y-y0
    
    a=np.mean(x2*y2)/np.mean(x2**2)
    
    b=y0-a*x0
    return a,b

In [ ]:
data_temchin2008=[2.3089873546759883, 1.1335705639594416,
2.433649800608806, 1.1219195832734208,
2.5189887453848376, 1.1773955803812832,
2.6533794225047744, 1.309265969016895,
2.778489919723694, 1.4971674771258776,
2.9012927263226755, 1.708120153194006,
3.0094886604141227, 1.9285965397126237,
3.1255960147906117, 2.1612852064604233,
3.2415506244104755, 2.2821647113582557,
3.357807354173958, 2.6241961613858074,
3.474104516373019, 2.9958241542560007,
3.586175253618712, 3.355814344419173,
3.7043916087598454, 3.5912389393077095,
3.8215714334900244, 3.8384495891670536,
3.939633359189716, 3.9608318329205066,
4.049971341946277, 4.16542819317891,
4.170808168162108, 4.435528842816696,
4.276243124454071, 4.561285269586788]
freqs_t=np.array(data_temchin2008[::2])
freqs_t=10**freqs_t
q10_t=np.array(data_temchin2008[1::2])



In [ ]:
bf_anf=[]
q10_anf=[]
with open('results/q101-22/q10s_clean.csv', 'r') as f:
    reader=csv.DictReader(f)
    for row in reader:
        bf_anf.append(float(row['CF'])*1e3)
        
        q10_anf.append(float(row['Q10']))
        

In [ ]:
freqs=np.array([1.5, 2.2, 3, 4, 5, 6, 8])*1e3
bw10=np.array([800, 2300, 600, 1000, 1300, 1400, 2600])
bw10_C=np.array([850,2400,850,1000,1250,2200,2500])


bw10_R=np.array([850,1450,350,1000,1250,950,2950])

pl.plot(freqs, freqs/bw10, '+', label='estimated w/ model')

pl.plot(freqs_t, q10_t, label='Temchin et al 2008')
pl.xlim([1000, 10000])
#pl.plot(freqs, freqs/bw10_C, '+', label='C')

#pl.plot(freqs, freqs/bw10_R, '+', label='R')

#pl.legend()

#lin reg
a_t, b_t = lin_reg(np.log(freqs_t), np.log(q10_t) )
a, b = lin_reg(np.log(freqs), np.log(freqs/bw10) )

f=np.linspace(1000, 10000)


#pl.plot(f, np.exp(b_t)*f**a_t)
#pl.plot(f, np.exp(b)*f**a)


pl.plot(bf_anf, q10_anf, 'x', label='ANF data')

pl.xscale('log')

pl.xlabel('f (Hz)')
pl.ylabel('Q10')
pl.legend(loc='lower right')
#pl.savefig('q10_estim.png')

pl.ylim([0.5, 7])
pl.show()

#pl.yscale('log')

### Estimations with optim

#### I/O funcs

In [ ]:
expe_name='4-23'  #'1-22'
Q10_distributed=True
CFs=[3000, 4000, 5000, 6000, 8000]


if Q10_distributed:
    results_folder=f'./results/fit{expe_name}-distrib/'
else:
    results_folder=f'./results/fit{expe_name}/'
    
CF=6000

In [ ]:
wb_cdf=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_1st_estim_{CF}.npz')
wb_cdf2=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_{CF}.npz')

In [ ]:
with np.load(f'{results_folder}/maskamountCAP_{CF}.npz') as data:
    I_pts=data['I_pts']
    maskamount=data['maskamount']
    


In [ ]:
I=np.linspace(-30, 20)


#pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')
pl.plot(I, wb_cdf(torch.tensor(I)), label='fit wbcdf')

pl.plot(I, wb_cdf2(torch.tensor(I)) , label='after optim')
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)

pl.plot(I0-20, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (max: broadband)')
pl.legend()

In [ ]:
I=np.linspace(-30, 20)

for CF in CFs:
    wb_cdf=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_{CF}.npz')

    pl.plot(I, wb_cdf(torch.tensor(I)), label=f'fit {CF}')
    
pl.xlabel('Power spectral density (dB)')

pl.ylabel('masking (max: broadband)')
pl.legend()

#### Unitary responses

In [ ]:

for CF in CFs:
    with np.load(f'{results_folder}/ur_{CF}.npz') as data:
        ur=data['ur']
        t2=data['t2']
        pl.plot(t2*1e3, ur, label=f"CF={CF} Hz")
        pl.xlim([3,8])
pl.xlabel('t (ms)')
pl.title('Unitary responses')
pl.legend()
pl.show()

In [ ]:

for CF in CFs:
    with np.load(f'{results_folder}/ur_{CF}.npz') as data:
        ur=data['ur']
        t2=data['t2']
        pl.plot(t2*1e3, ur/np.amax(np.abs(ur)), label=f"CF={CF} Hz")
        pl.xlim([3,8])
pl.xlabel('t (ms)')
pl.title('Unitary responses (normalized re:max)')
pl.legend()
pl.show()

#### E0

In [ ]:

for CF in CFs: #8000
    with np.load(f'{results_folder}/E0_{CF}.npz') as data:
        f=data['f']
        E0=data['E0']
        #TODO account for E0_amp
        #E0_amp=data['E0_amp']
        f=np.linspace(f[0], f[-1], len(E0)) #HACK err for now
        pl.plot(f, E0, label=f"CF={CF} Hz")
pl.legend()
pl.show()

Latencies

In [ ]:
ax=None
for CF in CFs: #8000
    lat_model=PowerLawLatencies.load_from_npz(f'{results_folder}/lat_{CF}.npz')
    lat_model.name=f'CF {CF}'
    ax=plotLatencies(lat_model, ax=ax)

Equivalent E0 in time

$w'(\tau) d\tau = w(f) A \alpha \tau^{\alpha-1} df$

In [ ]:
t=lat_model.t_from_f(f).detach().numpy()
A=lat_model.A.detach().numpy()
alpha=lat_model.alpha.detach().numpy()
t0=lat_model.t0.detach().numpy()
w=E0*A*alpha*np.power(np.abs(t-t0), alpha-1)
w=np.abs(w)


pl.plot(t*1e3, w*1e-8)
pl.xlim([5.5, 7])

for CF in CFs:
    t_CF=lat_model.t_from_f(CF).detach().numpy()
    ind=int((CF-f[0])/f[-1]*len(E0))
    w_CF=E0[ind]*A*alpha*np.power(np.abs(t_CF-t0), alpha-1)
    w_CF=np.abs(w_CF)
    dx=0.01
    dy=0.1*np.amax(w*1e-8)
    pl.arrow(t_CF*1e3+1.5*dx, w_CF*1e-8+1.5*dy, -dx, -dy, head_width=0.02, head_length=0.01,
             fc='k', ec='k', head_starts_at_zero=True)
    pl.text(t_CF*1e3+1.2*dx, w_CF*1e-8+1.2*dy, f'{CF} Hz')
pl.xlabel('t (ms)')
#pl.ylim([0,1])

#### Q10

#TODO single values

In [ ]:
if Q10_distributed:
    with np.load(f'{results_folder}/Q10.npz') as data:
        pl.plot(data['f'], data['Q10'], label=f'estim RBF {expe_name}')
        pl.ylabel('Q_10')
        pl.xlabel('f (Hz)')
        pl.xlim([1000, 9000])
        pl.ylim([0,8])

q10_gridsearch=[]
for CF in CFs:
    with np.load(f'{results_folder}/Q10gridsearch_{CF}.npz') as data:
        q10_gridsearch.append(CF/data['bw10_est'])
pl.plot(CFs, q10_gridsearch, '+', label=f'grid search {expe_name}')
        
pl.plot(bf_anf, q10_anf, 'x', label='ANF data 1-22')

pl.plot(freqs_t, q10_t, label='Temchin et al 2008')
pl.legend()

####  Errors

In [ ]:

for CF in CFs:
    err= np.load(f'{results_folder}/err_list_{CF}.npy', allow_pickle=True)
    pl.plot(err, label=f'{CF} Hz')
#pl.ylim([0.001, 0.006])
pl.legend()